In [201]:
\l cureOrig.q
sample:readsam[SIMPLE_SAMPLES[`:SAMPLE_SIMPLE1]]`
/show sample:readsam[FCPS_SAMPLES[`:SAMPLE_CHAINLINK]]`
/sample:readsam[FCPS_SAMPLES[`:SAMPLE_TARGET]]`

In [202]:
show sample:sample a,((til count sample) except a:sample?((asc sample)(floor(count sample)%2)))
dim:1_(til (first distinct count each sample)+1),1

6.554487 7.498119
3.522979 5.487981
3.768699 5.364477
3.423602 5.4199  
3.803905 5.389491
3.93669  5.663041
6.968136 7.755556
6.750795 7.269541
6.593196 7.850364
6.978178 7.60985 


In [2]:
dsc:{[n;r;s](r+n?s-r)*/:(cos;sin)@\:4*n?acos 0}
data :flip(-5 1)+(1 1.5)*dsc[1000; 0;1.8],'dsc[2000;3.1;4.2],'dsc[4000;5.2;6.5]
data,:flip(4 -1)+(1 8)*dsc[4000; 0;1.]
data@:neg[2000&count data]?count data
sample:data

In [203]:
/insert cluster into tree

insertKd:{[kd;sample2;L;clust] 
   /check if its to the left or right of initial cluster in tree
 
 dir:{(first x[0]`idx)>=0}{[sample2;kd;x] a:x[0];
  $[first (sample2[first x[0]`dim]>(first x[0]`rDim));
  i:update left:enlist 0,right:enlist 1 from exec idx,dim,rep,rDim from kd where right=1,parent=first a`idx;
  i:update left:enlist 1,right:enlist 0 from exec idx,dim,rep,rDim from kd where left=1,parent=first a`idx];
  (i;a)}[sample2;kd]/(i;i:exec from kd where parent=0,left=0,right=0); /insert cluster into the tree by looking at splitting dimension of each node &going left or right

 
 i:dir[0];
 a:dir[1];
 dim:((first a`dim)+1)mod 2; /get its new splitting dimension


 root:kd upsert flip update idx:(max kd`idx)+1,initi:L,clust:clust,rep:enlist sample2,
    rDim:enlist sample2[dim],dim:enlist dim,valid:enlist 1b,
    parent:enlist first a`idx from exec left,right from i; /update the info of new node into tree
 root}

In [5]:
/Calculating distances between clusters using kd tree. 


distC:{[kd;pt] 
   
 distCalc:{[kd;query;bestD]
   
 X:bestD[3]; / nodes that were already searched, not to be searched again
    
 cl:bestD[5]; / what cluster the node belongs to, as not to search any points in the same cluster
 n:bestD[1]; /nodes to search
 a:n where {[cl;kd;x](first exec clust from kd where idx=x)<>cl}[cl;kd]each n; /nodes to search that arent in the same cluster
  
 newD:imins,a i?imins:min i:{[kd;query;x] 
    sum m*m:(raze exec rep from kd where idx=x)-query}[kd;query] each a; /get minimum dist of all searched nodes

  $[(newD[0]<bestD[0])&(count a)<>0;(bestD[0]:newD[0];bestD[2]:newD[1])
     ;(bestD:bestD)]; /if new dist is less than current best dist, then that becomes new best dist
  
  axisD:(raze {[kd;bestD;query;x] $[(m*m:
    (first exec rDim from kd where idx=x)-
    query(first exec dim from kd where idx=x))<=bestD[0];
    (exec idx from kd where parent=x),exec parent from kd where idx=x;
    $[(query(first exec dim from kd where idx=x))<
    first exec rDim from kd where idx=x;
    (exec idx from kd where parent=x,left=1),exec parent from kd where idx=x;
    (exec idx from kd where parent=x,right=1),exec parent from kd where idx=x]]
    }[kd;bestD;query]each n)except bestD[3]:X,n; /get dists between node and search pts based on splitting dimension.
    /if =< than best Dist, than search the children of that node &parents.
    /Go up the tree and to the left/right based on whether that pt is <or> search pt
  
 (bestD[0];distinct axisD;bestD[2];bestD[3];bestD[4];cl)};
 dist:{[distCalc;kd;X] ({(count x[1])<>0}distCalc[kd;
    raze exec rep from kd where idx=X]/(0W;
    (raze (exec idx from kd where parent=X),exec parent from kd where idx=X)except X;X;X;X
    ;first exec clust from kd where idx=X))
    }[distCalc;kd;pt];
    
    kdC:update closDist:dist[0],closIdx:dist[2] from kd where idx=pt; /update new closDist and idx in tree
    kdC}

In [57]:
/Calculating distances between clusters using kd tree. 


distC:{[kd;pt] 
 kd:select from kd where valid:1b;
 distCalc:{[kd;query;bestD]
   
 X:bestD[3]; / nodes that were already searched, not to be searched again
    
 cl:bestD[5]; / what cluster the node belongs to, as not to search any points in the same cluster
 n:bestD[1]; /nodes to search
 a:n where {[cl;kd;x](first exec clust from kd where idx=x)<>cl}[cl;kd]each n; /nodes to search that arent in the same cluster
  
 newD:imins,a i?imins:min i:{[kd;query;x] 
    sum m*m:(raze exec rep from kd where idx=x)-query}[kd;query] each a; /get minimum dist of all searched nodes

  $[(newD[0]<bestD[0])&(count a)<>0;(bestD[0]:newD[0];bestD[2]:newD[1])
     ;(bestD:bestD)]; /if new dist is less than current best dist, then that becomes new best dist
  
  axisD:(raze {[kd;bestD;query;x] $[(m*m:
    (first exec rDim from kd where idx=x)-
    query(first exec dim from kd where idx=x))<=bestD[0];
    (exec idx from kd where parent=x),exec parent from kd where idx=x;
    $[(query(first exec dim from kd where idx=x))<
    first exec rDim from kd where idx=x;
    (exec idx from kd where parent=x,left=1,valid=1b),exec parent from kd where idx=x,valid:1b;
    (exec idx from kd where parent=x,right=1,valid:1b),exec parent from kd where idx=x,valid:1b]]
    }[kd;bestD;query]each n)except bestD[3]:X,n; /get dists between node and search pts based on splitting dimension.
    /if =< than best Dist, than search the children of that node &parents.
    /Go up the tree and to the left/right based on whether that pt is <or> search pt
  
 (bestD[0];distinct axisD;bestD[2];bestD[3];bestD[4];cl)};
 dist:{[distCalc;kd;X] ({(count x[1])<>0}distCalc[kd;
    raze exec rep from kd where idx=X]/(0W;
    (raze (exec idx from kd where parent=X,valid=1b),exec parent from kd where idx=X,valid=1b)except X;X;X;X
    ;first exec clust from kd where idx=X,valid=1b))
    }[distCalc;kd;pt];
    
    kdC:update closDist:dist[0],closIdx:dist[2] from kd where idx=pt; /update new closDist and idx in tree
    kdC}

In [6]:
/ delete from kd tree

deleteN:{[kd;X] 
 n:first exec idx from kd where initi=X;
 delN:{
  kd:x[0]; /kdtree
  X:x[1]; /point to be deleted
 
  delNode:select from kd where idx=X; /details of deleted pt
  axis:delNode`dim; /splitting dim

  mindim:$[(count exec idx from kd where right=1,parent=X)=0;
    raze {[kd;x](count exec idx from kd where
       parent=first x)<>0}[kd]{[kd;x]raze exec idx from kd where parent=last x}[kd]\
    first exec idx from kd where parent=X,left=1; / if has no right child then left child replaces 
    raze {[kd;x] (count exec idx from kd where
       parent=first x)<>0}[kd]{[kd;x]raze exec idx from kd where parent=last x}[kd]\
    first exec idx from kd where parent=X,right=1]; / get all the right children if there

  newP:mindim repPt?min repPt:(raze{[kd;x] exec rep from kd where idx=x}[kd]each mindim)axis; /the min value of rep pts from mindim based on splitting dimension
  newNode:select from kd where idx=newP; /get info from kdtree of newP
  tree:update rep:newNode`rep,initi:newNode`initi,closDist:newNode`closDist,
    clust:newNode`clust,clustIdx:newNode`clustIdx,rDim:(first newNode`rep)axis,
    closIdx:newNode`closIdx from kd where idx=X; /newNode replaces the deleted pt info in the tree, but dim and idx stay the same as deleted pt
  

  tree:update closIdx:X from tree where closIdx=first newNode`idx; /any pt that had the replacing pt as its closest idx has to update closIdx to its new value in the tree
  

    (tree;newP)};

 delCl:{(count select from (first x) where parent=last x)<>0}delN/(kd;n); /repeat this until reach a node with no children
 
 
 delete from first delCl where idx=last delCl} /delete the node with no children

In [110]:
/ delete from kd tree

deleteN:{[kd;X] 
 n:first exec idx from kd where initi=X;
 delN:{
  kd:x[0]; /kdtree
  X:x[1]; /point to be deleted
  delNode:select from kd where idx=X; /details of deleted pt
  axis:delNode`dim; /splitting dim

  mindim:$[(count exec idx from kd where right=1,parent=X,valid)=0;
    raze {[kd;x](count exec idx from kd where
       parent=first x,valid)<>0}[kd]{[kd;x]raze exec idx from kd where parent=last x,valid}[kd]\
    first exec idx from kd where parent=X,left=1,valid; / if has no right child then left child replaces 
    raze {[kd;x] (count exec idx from kd where
       parent=first x,valid)<>0}[kd]{[kd;x]raze exec idx from kd where parent=last x,valid}[kd]\
    first exec idx from kd where parent=X,right=1,valid]; / get all the right children if there

  newP:mindim repPt?min repPt:(raze{[kd;x] exec rep from kd where idx=x}[kd]each mindim)axis; /the min value of rep pts from mindim based on splitting dimension
  newNode:select from kd where idx=newP,valid; /get info from kdtree of newP
  tree:update rep:newNode`rep,initi:newNode`initi,closDist:newNode`closDist,
    clust:newNode`clust,clustIdx:newNode`clustIdx,rDim:(first newNode`rep)axis,
    closIdx:newNode`closIdx from kd where idx=X,valid; /newNode replaces the deleted pt info in the tree, but dim and idx stay the same as deleted pt
  

  tree:update closIdx:X from tree where closIdx=first newNode`idx; /any pt that had the replacing pt as its closest idx has to update closIdx to its new value in the tree
  

    (tree;newP)};

 delCl:{(count select from (first x) where parent=last x,valid)<>0}delN/(kd;n); /repeat this until reach a node with no children
 
 
 update valid:0b from first delCl where idx=last delCl} /delete the node with no children

In [47]:
/Create tree, search init nearest neighbours
createTree:{[sample]

 root:flip `idx`initi`rep`left`right`dim`parent`rDim`clust`clustIdx`valid!
   (0;0;enlist sample[0];enlist 0;enlist 0;enlist 0;enlist 0;enlist sample[0;0];0;enlist (0 0);1b); /insert first cluster

 kds:insertKd/[root;(1_sample);1_til count sample;1_til count sample]; /insert the rest of the clusters

 kds:update clustIdx:enlist each til count sample from kds; /insert the cluster indices

 kds:distC/[kds;kds`initi]; /get closest cluster to each cluster
 kds
 }

In [198]:
/Create tree, search init nearest neighbours


root:flip `idx`initi`rep`left`right`dim`parent`rDim`clust`clustIdx`valid!
   (0;0;enlist sample[0];enlist 0;enlist 0;enlist 0;enlist 0;enlist sample[0;0];0;enlist (0 0);enlist 1b);

kds:insertKd/[root;(1_sample);1_til count sample;1_til count sample]

kds:update clustIdx:enlist each til count sample from kds

show kds:distC/[kds;kds`initi]

idx initi rep               left right dim parent rDim     clust clustIdx val..
-----------------------------------------------------------------------------..
0   0     6.554487 7.498119 0    0     0   0      6.554487 0     0        1  ..
1   1     3.522979 5.487981 1    0     1   0      5.487981 1     1        1  ..
2   2     3.768699 5.364477 1    0     0   1      3.768699 2     2        1  ..
3   3     3.423602 5.4199   1    0     1   2      5.4199   3     3        1  ..
4   4     3.803905 5.389491 0    1     1   2      5.389491 4     4        1  ..
5   5     3.93669  5.663041 0    1     0   1      3.93669  5     5        1  ..
6   6     6.968136 7.755556 0    1     1   0      7.755556 6     6        1  ..
7   7     6.750795 7.269541 1    0     0   6      6.750795 7     7        1  ..
8   8     6.593196 7.850364 0    1     0   6      6.593196 8     8        1  ..
9   9     6.978178 7.60985  0    1     1   7      7.60985  9     9        1  ..


In [50]:
createTree sample

idx initi rep                  left right dim parent rDim       clust clustId..
-----------------------------------------------------------------------------..
0   0     -1.567473 -1.710713  0    0     0   0      -1.567473  0     0      ..
1   1     -1.323867 6.745647   0    1     1   0      6.745647   1     1      ..
2   2     -10.64619 -3.684479  1    0     1   0      -3.684479  2     2      ..
3   3     4.064173  5.552103   1    0     0   1      4.064173   3     3      ..
4   4     -5.231518 -7.445346  1    0     0   2      -5.231518  4     4      ..
5   5     0.7076635 4.846244   1    0     1   3      4.846244   5     5      ..
6   6     4.10755   -0.6628848 0    1     1   3      -0.6628848 6     6      ..
7   7     -2.231592 8.533084   0    1     0   2      -2.231592  7     7      ..
8   8     4.009992  -0.6682384 1    0     0   5      4.009992   8     8      ..
9   9     -4.043703 6.258881   1    0     1   7      6.258881   9     9      ..
10  10    -2.527246 8.824091   0    1   

In [40]:
clust:{[sample;numR;com;kd]
 j:first select from kd where closDist=first min closDist; /get min dist from tree
 
 j2:select clust from kd where idx=first j`closIdx; /get cluster of the closestPt
 
 old:select from kd where clust in ((j2`clust),j`clust); /get info of two clostest pts

 j0:(exec initi from kd where closIdx in old`idx); /get the initial indexes of those pts
 
 mean:avg pts:sample idxs:(distinct raze old`clustIdx);
 
 deleteClust:deleteN/[kd;idxs]; /delete merged clusters from tree
 
 $[(count pts)<=numR;(rep:pts;sami:idxs) ;rep:distinct sample sami:numR{[sample;idxs;x] x,maxIdx imax 
  {[sample;maxMean;x] min{[sample;x;maxMean]sum x*x:sample[maxMean]-sample[x]
     }[sample;x] each maxMean}[sample;x]each maxIdx:idxs except x
  }[sample;idxs]/maxFromMean:idxs imax sum each{x*x}mean-/:pts];
  /get rep pts based on the most spread out pts in cluster, initial pt i furthest from mean

 rep:(rep*1-com)+\:com*mean; /move rep pts based on compressed
 newCl:{[rep;x]min (sum each x*x:rep-\:x)}[rep]each deleteClust`rep; /dist of new cluster to all other clusters
 j3:(deleteClust`idx)[n:where newCl<deleteClust`closDist]; /any clusters where newCl<closDist

 closCl: (deleteClust`idx)where newCl=closD:min newCl; /what cluster is closest to the new clusters
 insertClust:insertKd/[deleteClust;rep;sami;first idxs]; /insert new rep pts into kdtree
 insertClust2:{[n;j3;newCl;insertClust;x] update closDist:newCl(n x),closIdx:last (insertClust`idx) from 
    insertClust where idx=j3(x)}[n;j3;newCl]/[insertClust;til count j3]; /update the tree with j3

 clustDist:update closIdx:first closCl,closDist:closD from insertClust2 where clust=first idxs; /update closDist of new clusters

 j5:exec idx from clustDist where clust=first idxs; /indexes of new clusters

  j4:(exec idx from clustDist where initi in j0) except j5; /any clusters that had the merged clusters as closDist
  recalc:distC/[clustDist;j4]; /recalc those distances
  insertIdx:enlist idxs; /initial indexes of merged cluster
 
  kd:{[insertIdx;kd;x] update clustIdx:(insertIdx) from kd where initi=x}[insertIdx]/[recalc;sami]; /update into kdtree
 
  kd}



In [10]:
cure:{[sample;numR;com;numCl] 
 cureTab:{[numCl;kd] (count distinct kd`clust)>numCl}[numCl]clust[sample;numR;com]/createTree sample;
 distinct cureTab`clustIdx}

In [45]:
\ts cure[sample;50;0;4]

54445 1591296


In [ ]:
show r:distinct a`clustIdx

/show r:distinct clust`clustIdx

In [ ]:
{plt[`:scatter][;]. flip sample[x]}each r;
plt[`:show][];

In [ ]:
a:(select rep by clust from clust)
show count b:exec rep from a
{plt[`:scatter][;]. flip x}each b;
plt[`:show][];

In [200]:
/insert cluster into tree

insertKd:{[kd;sample2;L;clust] 
   /check if its to the left or right of initial cluster in tree
 
 dir:{(first x[0]`idx)>=0}{[sample2;kd;x] a:x[0];
  $[first (sample2[first x[0]`dim]>(first x[0]`rDim));
  i:update left:enlist 0,right:enlist 1 from exec idx,dim,rep,rDim from kd where right=1,parent=first a`idx,valid;
  i:update left:enlist 1,right:enlist 0 from exec idx,dim,rep,rDim from kd where left=1,parent=first a`idx,valid];
  (i;a)}[sample2;kd]/(i;i:exec from kd where parent=0,left=0,right=0); /insert cluster into the tree by looking at splitting dimension of each node &going left or right

 
 i:dir[0];
 left:i`left;
 right:i`right;
 a:dir[1];
 dim:((first a`dim)+1)mod 2; /get its new splitting dimension
 b:exec idx from kd where valid=0b;
 root:update idx:(max kd`idx)+1, initi:L,clust:clust,rep:enlist sample2,
    valid:enlist 1b,dim:dim, rDim:sample2[dim], left:left,right:right,
    parent:a`idx from kd where idx=first b;  /update the info of new node into tree
 
 root}

In [200]:
numR:5
com:0
numCl:2

\ts clust:{[numCl;kd] (count distinct kd`clust)>numCl}[numCl]{[sample;numR;com;kd]
 j:first select from kd where closDist=first min closDist;
 
 j2:select clust from kd where idx=first j`closIdx;
 /show select idx, closDist,closIdx,clust from kd where closIdx=650;
 
 old:select from kd where clust in ((j2`clust),j`clust);
 j0:(exec initi from kd where closIdx in old`idx);
 
 mean:avg pts:sample idxs:(distinct raze old`clustIdx);
 
 show deleteClust:deleteN/[kd;idxs];
 
 maxFromMean:idxs imax sum each{x*x}mean-/:pts;
 
 rep:distinct sample sami:numR{[sample;idxs;x] x,maxIdx imax {[sample;maxMean;x] min{[sample;x;maxMean]sum x*x:sample[maxMean]-sample[x]
     }[sample;x] each maxMean}[sample;x]each maxIdx:idxs except x}[sample;idxs]/maxFromMean;
 
 rep:(rep*1-com)+\:com*mean;
 a:{[rep;x]min (sum each x*x:rep-\:x)}[rep]each deleteClust`rep;
 j3:(deleteClust`idx)[n:where a<deleteClust`closDist];
 b: (deleteClust`idx)where a=c:min a;
 show insertClust:insertKd/[deleteClust;rep;sami;first idxs];
 insertClust2:$[(count j3)=0;insertClust;{[n;j3;a;insertClust;x] update closDist:a(n x)
  ,closIdx:enlist (max (insertClust`idx)) from 
    insertClust where idx=j3(x)}[n;j3;a]/[insertClust;til count j3]];
 clustDist:update closIdx:first b,closDist:c from insertClust2 where clust=first idxs;
 /clustDist:distC/[insertClust2; j5:exec idx from insertClust2 where clust=first idxs];
 j5:exec idx from insertClust2 where clust=first idxs;
 j4:(exec idx from clustDist where initi in j0) except j5;

 recalc:distC/[clustDist;j4];
 insertIdx:enlist idxs;
  show kd:{[insertIdx;kd;x] update clustIdx:insertIdx from kd where initi=x}[insertIdx]/[recalc;sami];
  kd}[sample;numR;com]/kds

idx initi rep               left right dim parent rDim     clust clustIdx val..
-----------------------------------------------------------------------------..
0   0     6.554487 7.498119 0    0     0   0      6.554487 0     0        1  ..
1   1     3.522979 5.487981 1    0     1   0      5.487981 1     1        1  ..
2   3     3.423602 5.4199   1    0     0   1      3.423602 3     3        1  ..
3   3     3.423602 5.4199   1    0     1   2      5.4199   3     3        0  ..
4   4     3.803905 5.389491 0    1     1   2      5.389491 4     4        0  ..
5   5     3.93669  5.663041 0    1     0   1      3.93669  5     5        1  ..
6   6     6.968136 7.755556 0    1     1   0      7.755556 6     6        1  ..
7   7     6.750795 7.269541 1    0     0   6      6.750795 7     7        1  ..
8   8     6.593196 7.850364 0    1     0   6      6.593196 8     8        1  ..
9   9     6.978178 7.60985  0    1     1   7      7.60985  9     9        1  ..
idx initi rep               left right d

[0;31mlength[0m: [0;31mlength[0m

In [77]:
numR:10
com:0
numCl:4

\ts clust:{[numCl;kd] (count distinct kd`clust)>numCl}[numCl]{[sample;numR;com;kd]
 j:first select from kd where closDist=first min closDist; /get min dist from tree
 
 j2:select clust from kd where idx=first j`closIdx; /get cluster of the closestPt
 
 old:select from kd where clust in ((j2`clust),j`clust); /get info of two clostest pts
 j0:exec initi from kd where closIdx in old`idx; /get the initial indexes of those pts
 
 mean:avg pts:sample idxs:(distinct raze old`clustIdx); 
  

 deleteClust:deleteN/[kd;idxs]; /delete merged clusters from tree
 show select from deleteClust where valid=0b;
 maxFromMean:idxs imax sum each{x*x}mean-/:pts; /get pt with max dist from mean of cluster
 
 rep:distinct sample sami:numR{[sample;idxs;x] x,maxIdx imax {[sample;maxMean;x] min{[sample;x;maxMean
  ]sum x*x:sample[maxMean]-sample[x]
     }[sample;x] each maxMean}[sample;x]each maxIdx:idxs except x}[sample;idxs]/maxFromMean; 
 /get rep pts based on the most spread out pts in cluster
 
 rep:(rep*1-com)+\:com*mean; /move rep pts bsaed on compressed

 insertClust:insertKd/[deleteClust;rep;sami;first idxs]; /insert new rep pts into kdtree
 j4:exec idx from insertClust where initi in j0; /any clusters that had merged cluster as closest 
 
 clustDist:distC/[insertClust;distinct (exec idx from insertClust where clust=first idxs),j4]; /recalc dist of new inserted pts and j4  

 
 insertIdx:enlist idxs; /initial indexes of merged cluster
 kd:{[insertIdx;kd;x] update clustIdx:(insertIdx) from kd where initi=x}[insertIdx]/[clustDist;sami]; /update into kdtree
 
  kd}[sample;numR;com]/kds

idx initi rep             left right dim parent rDim   clust clustIdx valid c..
-----------------------------------------------------------------------------..
3   3     3.423602 5.4199 1    0     1   2      5.4199 3     3        0     0..


[0;31mtype[0m: [0;31mtype[0m